In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_KEY = os.getenv("api_key")
AZURE_DEPLOYMENT_NAME = os.getenv("model-name")
API_VERSION = os.getenv("api-version")
AZURE_OPENAI_ENDPOINT = os.getenv("azure_endpoint")

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# Create Azure Model Client
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=API_VERSION,
    deployment_name=AZURE_DEPLOYMENT_NAME ,  # Your deployment name
    model =AZURE_DEPLOYMENT_NAME 
)

In [ ]:
from autogen_core.models import UserMessage

result = await model_client.create([UserMessage(content="Tell me a joke.", source="user")])
print("OpenAI Response:", result.content)



In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken

In [ ]:
text_message = TextMessage(content="What is the current date and time?", source="user")

assistant = AssistantAgent("assistant", model_client=model_client)

response = await assistant.on_messages([text_message], cancellation_token=CancellationToken())
print(response.chat_message)

In [ ]:
# import required libraries
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
import datetime

In [ ]:
# Define a tool to get the current date and time
async def get_current_datetime() -> str:
    """Gets the current date and time."""
    
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_current_datetime],
    system_message="Use tools to solve tasks.",
)

In [ ]:
response = await agent.on_messages(
    messages=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Response:", response)
print("Final Response:", response.chat_message)
print("Inner Messages:", response.inner_messages)

In [ ]:
# Using run()
task_result = await agent.run(
    task=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Task Result:", task_result)
print("Final Response:", task_result.messages[-1].content)

In [ ]:
# Create a function which will be used as a tool
async def get_current_datetime() -> str:
    """Gets the current date and time."""
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")


In [ ]:
# import required libraries
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

from autogen_core.tools import FunctionTool
import datetime

# Function can be used as tool like below by directly specifying the function name
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_current_datetime],
    system_message="Use tools to solve tasks.",
)

# Or function can be used with tool with the help of FunctionTool
# This step happens automatically inside the AssistantAgent if the tool is a Python function.
get_datetime_tool = FunctionTool(get_current_datetime, description="Gets the current date and time")

agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_datetime_tool],
    system_message="Use tools to solve tasks.",
)

In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_datetime_tool],
    system_message="Use tools to solve tasks.",
    reflect_on_tool_use=True
)
response = await agent.on_messages(
    messages=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Final Response:", response.chat_message.content)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[get_datetime_tool],
    system_message="Use tools to solve tasks.",
    reflect_on_tool_use=True
)
response = await agent.on_messages(
    messages=[TextMessage(content="What is the current date and time?", source="user")],
    cancellation_token=CancellationToken(),
)
print("Final Response:", response.chat_message.content)

In [ ]:
import asyncio
from autogen_core import CancellationToken
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.messages import TextMessage


async def simple_user_agent():
    agent = UserProxyAgent("user_proxy")
    response = await asyncio.create_task(
        agent.on_messages(
            [TextMessage(content="What is your name? ", source="user")],
            cancellation_token=CancellationToken(),
        )
    )
    print(f"Your name is {response.chat_message.content}")

In [ ]:
await simple_user_agent()

In [ ]:
def burj_khalifa_height(prompt: str) -> str:
        return "The height of the Burj Khalifa is 100 ft"

In [ ]:

agent = UserProxyAgent(name="test_user", input_func=burj_khalifa_height)
messages = [TextMessage(content="What is the height of Burj Khalifa?", source="assistant")]

response = await agent.on_messages(messages, CancellationToken())

print(response.chat_message.content)

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
import PIL
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken

In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client
)

In [ ]:
image_path = "images/forest.png"  
pil_image = PIL.Image.open(image_path)
img = Image(pil_image)

multi_modal_message = MultiModalMessage(
    content=[
        "Please describe the general content of this image, focusing on the forest environment. Also, try to locate and describe the cat within the image.",
        img
    ],
    source="user"
)

img

In [ ]:
response = await agent.on_messages([multi_modal_message], CancellationToken())
print(response.chat_message.content)

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console

In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    model_client_stream=True,
)

In [ ]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.on_messages_stream(
    #     [TextMessage(content="Share information about azure ai search.", source="user")],
    #     cancellation_token=CancellationToken(),
    # ):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Share information about azure ai search.", source="user")],
            cancellation_token=CancellationToken(),
        ),
        output_stats=True,  # Enable stats printing.
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult

from autogen_agentchat.ui import Console


# Create the BlogPostDraftAgent
blog_post_draft_agent = AssistantAgent(
    "BlogPostDraftAgent",
    model_client=model_client,
    system_message="You are a blog post drafting agent. Your task is to generate an outline or initial draft of a blog post or article based on the topic provided. Make sure to include clear sections, headings, and a logical flow."
)

# Create the BlogPostEnhancementAgent
blog_post_enhancement_agent = AssistantAgent(
    "BlogPostEnhancementAgent",
    model_client=model_client,
    system_message="You are a blog post enhancement agent. Your task is to improve the draft provided by the drafting agent. Focus on improving readability, adding relevant examples, and optimizing for SEO. If you are satisfied with the final draft, respond with 'FINAL'."
)

# Define the termination condition that stops the task when 'FINAL' is mentioned
text_termination = TextMentionTermination("FINAL")

# Create a team where the BlogPostDraftAgent and BlogPostEnhancementAgent interact.
team = RoundRobinGroupChat(
    [blog_post_draft_agent, blog_post_enhancement_agent],
    termination_condition=text_termination
)

In [ ]:
# You can run a team using team.run() or team.run_stream()
async for message in team.run_stream(task="Create a short blog post about the benefits of using AI in e-commerce."): 
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

In [ ]:
await team.reset()  # Reset the team for the next run.

In [ ]:
await Console(team.run_stream(task="Create a short blog post about the benefits of using AI in e-commerce."))  # Stream the messages to the console.

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Suggest three creative ideas for a new mobile app.")
# Use asyncio.run(...) when running in a script.
await Console(stream)


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
assistant = AssistantAgent("assistant", model_client=model_client)

# Create the team setting a maximum number of turns to 1.
team = RoundRobinGroupChat([assistant], max_turns=1)

task = "Suggest three creative ideas for a new mobile app."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break


In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console




# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

In [ ]:
max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

await Console(round_robin_team.run_stream(task="Describe a futuristic city in 50 words"))

In [ ]:

# Define termination conditions
max_msg_termination = MaxMessageTermination(max_messages=10)
critic_approval_termination = TextMentionTermination(text="APPROVE")

# Combine conditions using OR (|)
combined_termination = max_msg_termination | critic_approval_termination

# Create a round-robin team with the combined termination condition
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Execute the conversation task
await Console(round_robin_team.run_stream(task="Provide a summary of the latest advancements in AI."))
